In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

/home/matt/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/matt/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
# Load movies metadata from movies_metadata.csv
movies_metadata = pd.read_csv('../movies_metadata.csv')
movies_metadata = movies_metadata.dropna(subset=['overview'])
movies_metadata['id'] = movies_metadata['id'].astype(int)
movies_metadata['overview'] = movies_metadata['overview'].astype('string')

In [3]:
# Load ratings data from ratings.csv
ratings_data = pd.read_csv('../ratings.csv')

In [27]:
# Group ratings by movieId and calculate average ratings
computed_ratings = ratings_data.groupby('movieId')['rating'].mean().reset_index()

In [36]:
# Merge movies metadata with average ratings using movieId as key
merged_data = pd.merge(movies_metadata, computed_ratings, left_on='id', right_on='movieId', how='inner')
print(merged_data.shape)

(82187, 26)


In [6]:
# Process overview text and obtain embeddings
overview_embeddings = model.encode(merged_data['overview'])

In [26]:
# Pair movieId with overview embeddings and average ratings
paired_data = pd.DataFrame({
    'movieId': merged_data['movieId'],
    'overview_embedding': overview_embeddings.tolist(),
    'average_rating': merged_data['average']
})

print(merged_data.shape)
print(paired_data.shape)

# Save paired data to a new CSV file
paired_data.to_csv('../paired_data.csv', index=False)

(7441, 27)
(7441, 3)


In [24]:
print(ratings_data.shape)
ratings_data.query('movieId == 2')
computed_ratings.query('movieId == 2')

(26024289, 4)


,movieId,rating,average
1,2,3.236953,3.24


In [20]:
movies_metadata.query('id == 2')

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
4342,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,1988-10-21,0,69.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}, {'iso_6...",Released,NaN,Ariel,False,7.1,44
